In [223]:
import pandas as pd
import numpy as np

import keras as keras
from keras.layers import Input, Dense, BatchNormalization
from keras.models import Model, load_model
from keras.losses import mean_squared_error as mean_squared_error_k
from keras import backend as K
from keras import regularizers


In [115]:
#import CCLE data
E_CCLE = pd.read_csv('E_CCLE.csv', index_col=0)
M_CCLE = pd.read_csv('M_CCLE.csv',index_col=0)


In [116]:
E_TCGA = pd.read_csv('E_TCGA.csv',index_col=0)

In [117]:
M_TCGA = pd.read_csv('M_TCGA.csv',index_col=0)

In [118]:
IC50 = pd.read_csv('IC50_CCLE_author.csv',index_col=0)

In [130]:
E_CCLE = E_CCLE.transpose()
E_CCLE.head()

,NIHOVCAR3,HL60,CACO2,HEL,HEL9217,MONOMAC6,LS513,C2BBE1,NCIH2077,253J,...,RC2,NB1643,TTC442,RH36,RH4,RH18DM,HCC827GR5,NCIH2882,JURKAT,A101D
ZBTB7A,2.765535,2.250962,2.063503,2.490570,2.232661,3.750607,3.649615,1.918386,2.508429,2.643856,...,1.752749,1.000000,1.144046,1.130931,1.815575,1.389567,2.867896,3.243364,2.684819,3.894333
CRH,0.163499,0.176323,0.056584,0.000000,0.028569,0.163499,0.000000,0.028569,0.014355,0.887525,...,0.000000,0.695994,0.000000,0.042644,0.000000,0.042644,0.163499,0.070389,0.000000,0.056584
TMEM74B,0.863938,0.042644,2.467279,1.454176,1.063503,0.137504,0.748461,1.811471,2.104337,1.454176,...,0.201634,1.344828,0.214125,0.389567,0.739848,0.389567,1.490570,0.014355,1.536053,0.000000
ERLIN2,4.254745,2.521051,4.303781,4.211791,4.180307,5.386811,4.943921,4.549669,4.899176,4.157852,...,4.628774,3.803227,4.963474,5.109361,4.847997,4.893848,4.198494,5.139961,4.187451,4.607034
ZBTB43,3.235727,1.613532,2.430285,2.384050,2.427606,3.949535,2.482848,2.675816,3.303050,2.372952,...,5.354029,3.104337,2.673556,1.839960,3.961623,2.472488,2.857981,2.090853,3.502076,2.625270


In [120]:
M_CCLE = M_CCLE.transpose()
M_CCLE.head()

,ZBTB7A,CRH,TMEM74B,ERLIN2,ZBTB43,POLL,SPANXN1,CPNE7,OR8B3,PKHD1L1,...,C11orf87,CUEDC2,ZNF799,FOXR1,BMPER,LAMA1,EPS15,GSC2,TP53BP1,MGAM
127399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
22RV1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0
2313287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,2.0
253J,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
M_TCGA = M_TCGA.transpose()
M_TCGA.head()

,ZBTB7A,CRH,TMEM74B,ERLIN2,ZBTB43,POLL,SPANXN1,CPNE7,OR8B3,PKHD1L1,...,C11orf87,CUEDC2,ZNF799,FOXR1,BMPER,LAMA1,EPS15,GSC2,TP53BP1,MGAM
0004d251-3f70-4395-b175-c94c2f5b1b81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000d566c-96c7-4f1c-b36e-fa2222467983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0011a67b-1ba9-4a32-a6b8-7850759a38cf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001887aa-36d0-463f-8bca-dec7043b4f2e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001944e5-af34-4061-9c09-bb9ea346f6fd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
E_TCGA = E_TCGA.transpose()
E_TCGA.head()

,ZBTB7A,CRH,TMEM74B,ERLIN2,ZBTB43,POLL,SPANXN1,CPNE7,OR8B3,PKHD1L1,...,C11orf87,CUEDC2,ZNF799,FOXR1,BMPER,LAMA1,EPS15,GSC2,TP53BP1,MGAM
d3b7ada6-5395-4212-a921-8e97c92b605d,4.256269,0.00000,0.722542,4.252478,2.521083,4.249446,0.000000,5.136277,0.0,0.201757,...,0.042784,5.354371,2.513489,0.00000,0.356247,1.131011,3.682567,0.0,3.874763,0.124460
065f7b7a-05ee-4728-bcfc-6095357512a6,3.670214,0.02871,4.086634,3.344838,2.508458,5.236450,0.000000,0.150686,0.0,0.956147,...,0.028710,6.134662,1.495726,0.00000,1.922249,0.201757,3.712564,0.0,4.603446,0.111167
065f7b7a-05ee-4728-bcfc-6095357512a6.1,4.084939,0.00000,3.150595,3.671320,2.963452,5.499225,0.000000,0.111167,0.0,0.433055,...,0.000000,6.549867,1.541050,0.00000,2.025058,0.992862,4.061800,0.0,4.644992,0.111167
c328f661-c109-4e01-b819-299a75bd4348,3.058325,0.00000,1.232706,5.463052,2.702668,4.407376,0.056724,0.632353,0.0,7.565331,...,2.241900,6.042239,1.275063,0.02871,0.555915,2.877778,4.200036,0.0,4.368048,0.000000
ebe927e6-0a7e-4c97-ad73-f3c302cca7bc,3.124356,0.00000,2.634650,4.100191,1.546046,4.803753,0.124460,3.927952,0.0,4.020564,...,0.014498,6.548086,2.153826,0.00000,0.632353,3.662199,3.561924,0.0,3.488046,0.298782


In [123]:
IC50.head()

,X22RV1,X2313287,X42MGBA,X5637,X639V,X647V,X697,X769P,X786O,X8305C,...,VCAP,VMCUB1,VMRCRCW,VMRCRCZ,WM115,WSUDLCL2,YAPC,YH13,YKG1,ZR7530
TL-2-105,4.6782,4.7229,4.1668,3.71110,4.4224,4.6946,1.19990,4.989000,4.1650,5.4715,...,4.2607,4.8047,4.53320,3.787200,3.5018,2.5862,5.5113,4.0055,4.61160,5.714937
TAK-715,4.7217,3.7747,5.5256,3.73420,4.9266,3.9165,1.60470,2.609200,3.1698,6.0290,...,4.6174,5.4785,4.14860,3.748600,5.1913,2.8146,5.7889,5.5701,3.70750,5.853699
CP466722,2.1995,2.9851,3.5194,1.80130,3.2444,2.0760,-0.86400,1.877600,2.1493,3.7663,...,1.8166,3.9553,2.80750,1.950200,1.9625,2.0836,3.6385,2.2756,2.41570,4.580076
BMS-345541,2.7886,3.4029,4.3641,2.67680,4.6885,3.3922,0.79615,1.491900,1.5246,5.1374,...,3.0436,4.6465,3.14600,3.087400,3.0976,2.4709,3.7157,3.5809,2.42940,4.263374
Genentech_Cpd_10,2.2161,2.8008,3.5257,-0.42507,2.1349,3.3500,-1.41520,-0.013295,-1.5783,5.3360,...,1.7366,2.5529,0.87061,-0.008367,1.8015,2.4950,5.6822,2.8236,0.91206,4.259721


In [124]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler



In [125]:
len(list(E_TCGA.index.values))

9327

In [241]:
class DummyScaler:
    def fit(self, x):
        self.data_min_ = np.zeros(x.shape[1],)
        self.data_max_ = np.ones(x.shape[1],)
    def transform(self, x): return x
    

scaler = StandardScaler()#MinMaxScaler()#DummyScaler() 
if scaler:
    scaler.fit(E_TCGA)
    E_TCGA_norm = pd.DataFrame(scaler.transform(E_TCGA), columns = E_TCGA.columns, index=E_TCGA.index)
    print (E_TCGA_norm['ALAS1'].mean(), E_TCGA_norm['ALAS1'].std())
    print(E_TCGA_norm['ALAS1'].describe())

1.5888884147130225e-14 1.0000536121163788
count    9.327000e+03
mean     1.588888e-14
std      1.000054e+00
min     -4.705958e+00
25%     -6.037910e-01
50%     -1.103349e-01
75%      4.394181e-01
max      7.390753e+00
Name: ALAS1, dtype: float64


In [127]:
#E_TCGA_norm = (E_TCGA-E_TCGA.mean())/E_TCGA.std() #Mean - 0 Std_dev-  1 
#E_TCGA_norm = (E_TCGA - E_TCGA.min()) / (E_TCGA.max() - E_TCGA.min()) #Values between 0 and 1

In [128]:
E_TCGA['ALAS1'].describe()

count    9327.000000
mean        5.186317
std         0.781032
min         1.511009
25%         4.714762
50%         5.100147
75%         5.529499
max        10.958425
Name: ALAS1, dtype: float64

In [242]:
#reserve a random sample of 2000 samples to validate the model
from random import sample 
valid_idx = sample(list(E_TCGA_norm.index.values),2000 )
x_valid =  E_TCGA_norm.loc[valid_idx, :]
x_train = E_TCGA_norm.drop(valid_idx, axis=0)
print (x_train.shape) 
print (x_valid.shape)
x_train.describe()

(7327, 17947)
(2000, 17947)


,ZBTB7A,CRH,TMEM74B,ERLIN2,ZBTB43,POLL,SPANXN1,CPNE7,OR8B3,PKHD1L1,...,C11orf87,CUEDC2,ZNF799,FOXR1,BMPER,LAMA1,EPS15,GSC2,TP53BP1,MGAM
count,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,...,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000,7327.000000
mean,0.009693,-0.000380,0.006575,0.000902,0.001601,0.003001,0.000220,0.009953,0.005609,-0.006840,...,-0.000134,-0.000056,0.000507,0.001017,-0.003382,-0.003273,0.001984,0.004558,-0.002147,0.004717
std,1.006901,0.994432,1.005821,0.997027,1.007557,1.003701,0.980103,1.008354,1.027576,0.990218,...,0.996895,1.002634,0.998395,1.009098,0.991424,1.000763,1.006319,1.048184,1.004097,1.007570
min,-3.463876,-0.224101,-1.445642,-3.681745,-3.134870,-4.882774,-0.151950,-1.364514,-0.125737,-0.583487,...,-0.325821,-4.898671,-2.730503,-0.200066,-0.750925,-1.087274,-3.972733,-0.240360,-4.073098,-0.512949
25%,-0.612778,-0.224101,-0.752912,-0.617464,-0.641920,-0.674657,-0.151950,-0.813741,-0.125737,-0.557638,...,-0.325821,-0.624563,-0.647131,-0.200066,-0.606318,-0.819710,-0.624434,-0.240360,-0.567393,-0.463369
50%,-0.031765,-0.224101,-0.242721,0.032530,0.047619,0.027605,-0.151950,-0.191196,-0.125737,-0.368382,...,-0.298873,0.033401,-0.041558,-0.200066,-0.388495,-0.352278,0.017806,-0.240360,0.070388,-0.353454
75%,0.539309,-0.189398,0.533483,0.627726,0.662155,0.715021,-0.151950,0.627348,-0.125737,0.098185,...,-0.246294,0.648430,0.612988,-0.200066,0.156898,0.627201,0.621695,-0.240360,0.657848,-0.074193
max,3.964197,13.413552,5.577861,6.729774,4.403603,3.677282,29.626407,4.546028,32.317783,7.454908,...,10.451065,4.115365,6.498394,14.465617,6.262715,4.283479,4.263484,36.575135,3.032769,7.443353


In [163]:
x_train.values[2,:100]


array([-0.60143198, -0.224101  ,  1.23015433, -0.16934295, -1.11478644,
        1.36576949,  1.54109185,  1.52567138, -0.12573675,  3.03639345,
       -0.40321354,  0.64817569,  0.08664087,  0.03905313, -0.87346493,
       -0.35348861, -0.71347249, -0.64677551,  0.30490414,  0.26958662,
       -0.05955466,  0.25128114,  0.82137919,  0.51169417,  0.2659441 ,
       -0.1566827 ,  0.81302147, -0.95344303, -0.19268109, -1.10825634,
       -0.40384554,  0.76446591,  1.25222788, -0.66915377, -0.10486425,
       -0.57814031, -0.11866222,  0.86400945,  0.6157417 , -0.34130126,
        0.04553338, -0.99195321,  0.86074894,  0.66635639,  0.7959031 ,
       -0.41670092,  0.72041842, -0.33437416,  0.43932365,  0.09403434,
        0.87248608,  0.01197119,  0.2139697 ,  0.34422521,  0.54293969,
       -0.61395205, -0.54113584, -0.28793543,  0.27388216, -0.22771071,
       -0.09295591, -0.54098464,  0.80187533,  0.13049861, -0.21947257,
       -0.50753111, -0.85491366, -0.19717264, -0.41004755, -0.48

In [132]:
#Train a gene expresion autoencoder with TCGA data (n_samples, n_genes), Eenc (n_genes, 1,024, 256, and 64)

In [164]:
encoding_dim = 64  # 64 floats -> compression of factor 343, assuming the input is 22374 floats

tumor_dim = E_TCGA.shape[1]
tumor_dim


17947

In [165]:
K.clear_session()
layers = [1024,256,128]

# this is our input placeholder
input_tumor = Input(shape=(tumor_dim,))
use_bn = False
# "encoded" is the encoded representation of the input
for i,layer in enumerate(layers):
    encoded = Dense(layer, activation='relu')(input_tumor if i == 0 else encoded)
    if use_bn: encoded = BatchNormalization()(encoded)
#encoded = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-5))(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

for i,layer in enumerate(layers[::-1]): 
    decoded = Dense(layer, activation='relu')(encoded if i == 0 else decoded)
    if use_bn: decoded = BatchNormalization()(decoded)
out = Dense(tumor_dim, activation='linear')(decoded)


In [166]:
# this model maps an input to its reconstruction
autoencoder = Model(input_tumor, out)
# this model maps an input to its encoded representation
encoder = Model(input_tumor, encoded)

In [167]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 17947)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              18378752  
_________________________________________________________________
dense_2 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_6 (Dense)              (None, 256)               33024     
__________

In [168]:
# create a placeholder for an encoded (64-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
#decoder = Model(encoded_input, decoder_layer(encoded_input))
#decoder = Model(encoded_input, decoder_layer(encoded_input))

In [224]:
def metric_mse(y_true, y_pred):
    if isinstance(scaler, StandardScaler):
        _y_true = y_true * scaler.scale_ + scaler.mean_
        _y_pred = y_pred * scaler.scale_ + scaler.mean_        
    else:
        _y_true = y_true * (scaler.data_max_ - scaler.data_min_) + scaler.data_min_
        _y_pred = y_pred * (scaler.data_max_ - scaler.data_min_) + scaler.data_min_
    return mean_squared_error_k(_y_true, _y_pred)
    

In [170]:
adam = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
autoencoder.compile(optimizer= adam, loss='mean_squared_error', metrics=[metric_mse])
#autoencoder.compile(optimizer= adam, loss='binary_crossentropy')


In [172]:
autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_valid, x_valid))

Train on 7327 samples, validate on 2000 samples
Epoch 1/100
7327/7327 [==============================] - 9s 1ms/step - loss: 0.6949 - metric_mse: 0.7393 - val_loss: 0.5456 - val_metric_mse: 0.5538
Epoch 2/100
7327/7327 [==============================] - 9s 1ms/step - loss: 0.5084 - metric_mse: 0.4882 - val_loss: 0.4524 - val_metric_mse: 0.4343
Epoch 3/100
7327/7327 [==============================] - 9s 1ms/step - loss: 0.4392 - metric_mse: 0.4082 - val_loss: 0.4162 - val_metric_mse: 0.3911
Epoch 4/100
7327/7327 [==============================] - 9s 1ms/step - loss: 0.4088 - metric_mse: 0.3739 - val_loss: 0.3959 - val_metric_mse: 0.3675
Epoch 5/100
7327/7327 [==============================] - 9s 1ms/step - loss: 0.3878 - metric_mse: 0.3517 - val_loss: 0.3814 - val_metric_mse: 0.3515
Epoch 6/100
7327/7327 [==============================] - 9s 1ms/step - loss: 0.3712 - metric_mse: 0.3344 - val_loss: 0.3704 - val_metric_mse: 0.3396
Epoch 7/100
7327/7327 [==============================] - 9

7327/7327 [==============================] - 9s 1ms/step - loss: 0.2405 - metric_mse: 0.2143 - val_loss: 0.3396 - val_metric_mse: 0.3025
Epoch 56/100
7327/7327 [==============================] - 9s 1ms/step - loss: 0.2419 - metric_mse: 0.2158 - val_loss: 0.3422 - val_metric_mse: 0.3045
Epoch 57/100
7327/7327 [==============================] - 9s 1ms/step - loss: 0.2438 - metric_mse: 0.2158 - val_loss: 0.3410 - val_metric_mse: 0.3035
Epoch 58/100
7327/7327 [==============================] - 9s 1ms/step - loss: 0.2400 - metric_mse: 0.2127 - val_loss: 0.3402 - val_metric_mse: 0.3027
Epoch 59/100
7327/7327 [==============================] - 9s 1ms/step - loss: 0.2379 - metric_mse: 0.2105 - val_loss: 0.3400 - val_metric_mse: 0.3025
Epoch 60/100
7327/7327 [==============================] - 9s 1ms/step - loss: 0.2364 - metric_mse: 0.2097 - val_loss: 0.3400 - val_metric_mse: 0.3023
Epoch 61/100
7327/7327 [==============================] - 9s 1ms/step - loss: 0.2348 - metric_mse: 0.2086 - val_l

In [219]:
autoencoder.save('model_encoder_E_TCGA_std.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model



In [243]:
p = autoencoder.predict(x_valid)
np.set_printoptions(suppress=True)
np.around(p[1][:100],decimals=8)

array([ 1.0857009 ,  0.06824583,  0.94729114, -0.27819464,  0.02031765,
        0.25912333, -0.22548953,  1.6036053 , -0.14870858, -0.10596123,
       -0.5652775 ,  1.2278466 , -0.00867097, -1.0842278 ,  0.8159287 ,
       -0.06325143, -0.67181253, -0.39295903,  0.36419675, -0.35738456,
       -1.0666223 ,  1.0985821 ,  1.1053166 ,  0.9156634 , -0.30579123,
       -0.23819244, -1.3039116 ,  0.5442993 , -0.21570754,  0.5543682 ,
       -0.43463165, -0.620371  , -0.45542783,  0.8378333 , -0.02170878,
        1.5418226 , -0.5942787 , -0.65844685,  1.7723801 , -0.50450635,
       -0.16782995,  0.6691558 , -0.06961058,  0.18973294,  1.233103  ,
        0.51065665, -1.081161  ,  0.7214816 , -0.35820872, -1.4884266 ,
        0.18780673, -0.07354928,  0.7202826 ,  0.6142722 ,  0.31557426,
       -0.07264862, -0.6306247 , -0.43787003, -0.42254308,  0.40754843,
       -0.2740486 , -0.9930557 , -0.8932992 ,  0.9787187 , -0.6586889 ,
       -1.2373184 , -0.30529714,  0.04399962, -0.573373  , -0.55

In [250]:
from sklearn.metrics import mean_squared_error
i = 10
y_true = x_valid.values
y_pred = p
print ('MSE scale non reverted')
print(mean_squared_error(y_true, y_pred))

def rescale(y_true, y_pred):
    if isinstance(scaler, StandardScaler):
        _y_true = y_true * scaler.scale_ + scaler.mean_
        _y_pred = y_pred * scaler.scale_ + scaler.mean_        
    else:
        _y_true = y_true * (scaler.data_max_ - scaler.data_min_) + scaler.data_min_
        _y_pred = y_pred * (scaler.data_max_ - scaler.data_min_) + scaler.data_min_
    return _y_true, _y_pred

_y_true, _y_pred = rescale(y_true, y_pred)


print ('MSE scale reverted')
print(mean_squared_error(_y_true, _y_pred))
print ('Example of reconstructed sample (true, pred)')
print(list(zip(_y_true[i], _y_pred[i])))

x_valid_original =  E_TCGA.loc[valid_idx, :]
y_true_original = x_valid_original.values
#print(mean_squared_error(y_true_original,_y_true))
#mean_squared_error(y_true_original,_y_pred)

MSE scale non reverted
0.24232213406541916
MSE scale reverted
0.21866313900272216
Example of reconstructed sample (true, pred)
[(2.39509677887, 2.616115687542038), (0.0, 0.40006371814783376), (0.42232954502100006, 0.448405411366969), (4.29354047775, 4.202734701290774), (1.29874289036, 1.7236507802225904), (3.9827773571, 4.240069463302671), (0.0, -0.018373681028390435), (3.97088313103, 2.878267516981059), (0.0, -0.03191676171651242), (0.201756611466, 0.6801867507857493), (0.08420307189230003, 0.10146719855195963), (5.67891359329, 6.036370679276383), (5.561939239500001, 5.809828370136553), (6.05681610107, 5.775943017779402), (6.25716876984, 6.330638655718493), (5.49218416214, 5.250539056591089), (0.0, 0.5181341883437196), (0.0, 0.06920357028372387), (7.01915550232, 6.615071262206714), (0.6041815876959999, 0.8021172488142472), (0.056723508983899995, 0.9056971339773822), (4.34132003784, 4.196582466211217), (1.0215282440200002, 1.7149368860555716), (5.8572807312, 5.45576905410783), (1.84806

In [245]:
x_valid.iloc[1,:100].values

array([ 0.79446131, -0.224101  ,  0.3329135 , -0.50821124,  0.49585691,
        0.08309032, -0.15195033,  1.233672  , -0.12573675, -0.12791866,
       -0.59962976,  1.06480776, -0.56852552, -0.86782644,  1.08451871,
        0.41865604, -0.46641535,  0.14889334,  0.27237021, -0.31060436,
       -1.08883595,  1.41359084,  1.40044085,  0.01211876, -0.21194254,
       -0.1566827 , -1.70652202,  0.04290158, -0.19268109,  0.39925229,
       -0.39158082, -0.73928893, -1.12290195,  0.85737805, -0.10486425,
        1.41418462, -0.46951688, -0.74462564,  1.99010997, -0.36327517,
       -0.21769532,  0.7137195 ,  0.32238122, -0.27900254,  1.39465624,
        0.12685601, -1.36714766,  0.31398167,  0.09819182, -2.20063631,
       -0.31554638,  0.64361745,  1.22867843,  1.50691518,  0.25428098,
       -0.40653707, -0.54113584, -0.20670703, -0.68358694,  0.34343007,
       -0.09295591, -0.94641527, -1.07739083,  0.73163636,  0.49701352,
       -1.96851667, -0.69109544, -0.19717264, -1.01274304, -0.49

In [240]:
autoencoder = load_model('model_encoder_E_TCGA_std.h5', custom_objects={'metric_mse':metric_mse})